In [77]:
import os, sys, re, io, pathlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import cudf
# import cuml
from cuml.manifold import TSNE
from cuml.cluster import KMeans
from sklearn.preprocessing import normalize

buffer = io.StringIO()

# define the current path (notebooks in lab_utils)
labutilspath = str(pathlib.Path(os.getcwd()).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

pp = autoscan.basics(material_info = True)

def ix_before_and_after(ds, index = ['tag', 'subtag'], columns = 'experiment', values = 'ix', mask = None, subset = None, aggfunc = lambda x: [*x],
                        dropna_tresh = 2, chain = True):
    ds.loc[:, 'ix'] = ds.index.values
    if mask is not None:
        ds = ds.loc[mask, :]
    dx = ds.pivot_table(index = index, 
                        columns = columns, 
                        values = values, 
                        aggfunc = aggfunc)
    if subset is not None:
        dx = dx.loc[:, subset]
    dx.dropna(thresh = dropna_tresh, inplace = True)

    # get the index of samples with before and after characterization
    if chain:
        ix = list(itertools.chain(*dx.apply(lambda x: [*itertools.chain(*[s for s in x if type(s) != float])], axis = 1).values))
    else:
        ix = dx
    return ix

def set_spe_style(ax, title = '', xlabel = '', ylabel=''):
    plt.sca(ax)
    plt.title(title, fontweight = 'bold');
    plt.xlabel(xlabel, fontweight = 'bold')
    plt.ylabel(ylabel, fontweight = 'bold');
    plt.xticks(fontweight = 'bold');
    plt.yticks(fontweight = 'bold');
    return ax

def kmeans_calc(data, nclusters = 4, kmeans_iter = 10000, kmeans_inplace = True, 
                plot = False, plot_save = True, plot_figsize = (12, 12), 
                plot_x = 'x', plot_y = 'cluster', plot_hue = 'c', plot_title = '', plot_savename = 'kmeans.png', plot_figpath = './'):
    df = data.copy()
    kmeans = KMeans(n_clusters = nclusters, max_iter = kmeans_iter, init = 'scalable-k-means++', n_init = 10)
    kmeans.fit(data.iloc[:, 5:])
    out = None
    df.loc[:, 'cluster'] = kmeans.labels_
    if kmeans_inplace:
        data.loc[:, 'cluster'] = kmeans.labels_
    else:
        out = kmeans.labels_
    
    if plot:
        fig, ax = plt.subplots(figsize = plot_figsize)
        sns.stripplot(x = plot_x, y = plot_y, hue = plot_hue, dodge = True, data = df, ax = ax)
        plt.yticks(range(nclusters), range(nclusters))
        plt.title(plot_title)
        if plot_save:
            plt.savefig(os.path.join(plot_figpath, plot_savename), dpi = 300, bbox_inches = 'tight')

    return out

def tsne_calc(data, datacols = None, perplexity = 30, n_iter = 10000, angle = 0.8, learning_rate = 10, random_state = 5, leftcols = None, concat = True):    
    p = perplexity
    neighbors = int(3 * p)
    if datacols is not None:
        v = data.loc[:, datacols].values
    else:
        v = data.values
    
    tsne = TSNE(n_components = 2, n_iter = n_iter, angle = angle, n_neighbors = neighbors, perplexity = p, 
                random_state = random_state, learning_rate = 10)
    X_hat = tsne.fit_transform(v)
    X_hat = pd.DataFrame(X_hat, columns = ['u', 'v'])
    
    out = None
    
    if concat:
        
        if leftcols is not None:
            left = data.loc[:, leftcols].copy()
        else:
            left = data
        out = pd.concat([left, X_hat], axis = 1)
    else:
        out = X_hat
    return out

def ftir_kmeans_plot():
    pass
#     x_hat = pd.concat([df_ftir.loc[:, left_cols], ], axis = 1)
#     fig, ax = plt.subplots(ncols = 2, figsize = (24, 13), sharey = True)
#     plt.subplots_adjust(wspace = 0.05)
#     sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'before'"), ax = ax[0], palette = 'viridis', legend = 'full')
#     sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'after' "), ax = ax[1], palette = 'viridis', legend = 'full')
#     ax[0].set_title('before')
#     ax[1].set_title('after')
#     fig.suptitle('tSNE before & after for all samples, perplexity = ' + str(p), fontsize = 12, fontweight = 'bold', y = 0.92)
#     plt.savefig(os.path.join(figspath, 'all-rocks_tsne.png'), dpi = 300, bbox_inches = 'tight')

rc_dict = {
    "font.size":12,
    'font.weight':'bold',
    "axes.titlesize":12,
    "axes.titleweight":'bold',
    "axes.labelsize":12,
    'axes.labelweight':'bold',
    'xtick.labelsize':12,
    'ytick.labelsize':12,
    'legend.frameon':True,
    'legend.fontsize':12,
    'legend.title_fontsize':12,
}

sns.set_context("paper", 
                rc = rc_dict)

sns.set(rc = rc_dict)

sns.set_style('darkgrid')

In [13]:
# define paths
datapath = '/sandbox/data/autoscan/'
vispath = '/sandbox/vis/autoscan/'
savepath = datapath
datafname = 'autoscan_corrected.h5'

datafile = os.path.join(datapath, datafname)
figspath = os.path.join(vispath, 'rock_multiphysics_display')

In [14]:
for p in [figspath, savepath]:
    if not os.path.exists(p):
        os.makedirs(p)

In [15]:
# read the data
dd = pd.read_hdf(datafile, key = 'data')
ds = pd.read_hdf(datafile, key = 'desc')
df = ds.join(dd)
del dd, ds

# df.columns = list(df.columns[:10].values) + list(pp.ftir_lambdas) + list(df.columns[-23:].values)
# set the value limits for perm
for k,v in {'sandstone':1e3, "shale":1e2, 'carbonate':1e3}.items():
    ix = df.loc[(df.family == k) & (df.instance == 'before'), 'perm'] > v
    fill_val = df.loc[ix.index[ix == False], 'perm'].mean()
    df.loc[ix.index[ix == True], 'perm'] = fill_val

df = df.query("family != 'metal' & family != 'gemstones'").copy()
df = df.sort_values(by ='instance', ascending = False, ignore_index = True)

In [16]:
# normalize(df.loc[df.l_1.isna() == False, pp.ftir_cols], norm = 'max')
df_temp = df.dropna(subset = pp.ftir_cols).copy()
df_temp.loc[:, pp.ftir_cols] = normalize(df_temp.loc[:, pp.ftir_cols], norm = 'max')
ix = df_temp.index
df.loc[ix, pp.ftir_cols] = df_temp.loc[:, pp.ftir_cols].values
del df_temp, ix

In [17]:
# df_ftir_bna = df.loc[ix, pp.meta_cols + pp.grid_cols + pp.ftir_cols]

In [27]:
df_ftir = df.loc[:, ['family', 'tag','instance', 'l_max_peak'] + pp.ftir_cols].dropna()
newcolumns = list(df_ftir.columns[:4].values) + list(pp.ftir_lambdas)
df_ftir.columns = newcolumns
df_ftir.reset_index(drop = False, inplace = True)
# df_ftir.iloc[:, 4:] = normalize(df_ftir.iloc[:, 4:])
df_ftir.head()
# df_ftir.iloc[:, 2:] = normalize(df_ftir.iloc[:, 2:], norm = 'max')

,index,family,tag,instance,l_max_peak,398.647,400.702,402.757,404.812,406.867,408.922,410.977,413.031,415.086,417.141,419.196,421.251,423.306,425.361,427.416,429.471,431.525,433.58,435.635,437.69,439.745,441.8,443.855,445.91,447.964,450.019,452.074,454.129,456.184,458.239,460.294,462.349,464.404,466.458,468.513,470.568,472.623,474.678,476.733,478.788,480.843,482.897,484.952,487.007,489.062,491.117,493.172,495.227,497.282,499.337,501.391,503.446,505.501,507.556,509.611,511.666,513.721,515.776,517.83,519.885,521.94,523.995,526.05,528.105,530.16,532.215,534.27,536.324,538.379,540.434,...,3844.686,3846.741,3848.796,3850.85,3852.905,3854.96,3857.015,3859.07,3861.125,3863.18,3865.235,3867.289,3869.344,3871.399,3873.454,3875.509,3877.564,3879.619,3881.674,3883.729,3885.783,3887.838,3889.893,3891.948,3894.003,3896.058,3898.113,3900.168,3902.222,3904.277,3906.332,3908.387,3910.442,3912.497,3914.552,3916.607,3918.662,3920.716,3922.771,3924.826,3926.881,3928.936,3930.991,3933.046,3935.101,3937.155,3939.21,3941.265,3943.32,3945.375,3947.43,3949.485,3951.54,3953.595,3955.649,3957.704,3959.759,3961.814,3963.869,3965.924,3967.979,3970.034,3972.088,3974.143,3976.198,3978.253,3980.308,3982.363,3984.418,3986.473,3988.528,3990.582,3992.637,3994.692,3996.747
0,0,shale,sh_001,before,406.867,0.947639,0.949593,0.976160,0.995882,1.000000,0.999916,0.991398,0.983236,0.976724,0.970515,0.975131,0.968315,0.948678,0.928772,0.904866,0.890060,0.891846,0.892634,0.882096,0.877506,0.878122,0.870638,0.859796,0.850939,0.841426,0.829414,0.818378,0.814845,0.812460,0.802754,0.794283,0.790090,0.784638,0.777766,0.774743,0.776820,0.778355,0.780397,0.785232,0.791547,0.799511,0.806093,0.807431,0.805974,0.806375,0.806463,0.802292,0.796255,0.793267,0.792479,0.790627,0.787719,0.785690,0.784163,0.781558,0.779218,0.777902,0.775755,0.771979,0.771280,0.775495,0.778742,0.777779,0.775482,0.774703,0.774210,0.774488,0.775574,0.774712,0.774215,...,0.892876,0.894548,0.896281,0.894847,0.891714,0.889026,0.889818,0.892594,0.893267,0.892885,0.892040,0.891732,0.892445,0.893527,0.894702,0.894420,0.893963,0.894970,0.895278,0.894156,0.893382,0.894037,0.894702,0.894108,0.893980,0.894508,0.893118,0.890478,0.889426,0.890143,0.890438,0.891490,0.894143,0.896528,0.897074,0.896273,0.896290,0.896906,0.896242,0.894913,0.894561,0.894957,0.894442,0.893835,0.895001,0.896871,0.897628,0.896805,0.895221,0.894504,0.894772,0.895850,0.897408,0.897725,0.896906,0.896788,0.897434,0.897813,0.898002,0.898367,0.898328,0.897949,0.897346,0.895789,0.894196,0.893958,0.894064,0.894147,0.894895,0.895371,0.895014,0.894429,0.894530,0.895415,0.895784
1,12943,shale,sh_001,before,400.702,0.993814,1.000000,0.925277,0.909910,0.935920,0.944133,0.913653,0.878019,0.860032,0.857239,0.857428,0.883235,0.878102,0.851242,0.837233,0.826899,0.806236,0.791397,0.793352,0.796560,0.788226,0.771190,0.757842,0.752011,0.742900,0.730004,0.722618,0.721875,0.720844,0.717980,0.714508,0.708998,0.703106,0.697313,0.691765,0.690418,0.695502,0.703899,0.709043,0.708832,0.708764,0.715557,0.723015,0.723267,0.722716,0.721558,0.716904,0.712477,0.708809,0.705677,0.703401,0.700427,0.695517,0.689905,0.687327,0.687199,0.687459,0.689459,0.692218,0.692211,0.690135,0.689135,0.689297,0.689878,0.690561,0.689210,0.686678,0.686036,0.685082,0.683640,...,0.774145,0.774255,0.777142,0.779320,0.778776,0.776032,0.773523,0.774119,0.775429,0.774859,0.772504,0.772043,0.775198,0.777946,0.778908,0.778738,0.776806,0.774783,0.774255,0.775285,0.776923,0.777538,0.777410,0.777176,0.776901,0.777452,0.778112,0.777244,0.775078,0.773149,0.771651,0.770873,0.773043,0.776285,0.777210,0.777335,0.778308,0.778271,0.777074,0.776278,0.776229,0.776667,0.777742,0.779029,0.780116,0.781048,0.781116,0.780622,0.779580,0.778455,0.778576,0.779267,0.779746,0.780018,0.779746,0.778780,0.778293,0.778508,0.778014,0.777565,0.777716,0.777369,0.777784,0.779214,0.780003,0.779999,0.779044,0.777667,0.776738,0.776014,0.774957,0.774300,0.774870,0.775821,0.776848
2,34079,shale,sh_001,before,402.757,0.923390,0.962308,1

In [ ]:
print("number of samples: %d, number of features: %d" %(df_ftir.shape))

In [ ]:
nclusters = 10
kmeans = KMeans(n_clusters = nclusters, max_iter = 10000, init = 'scalable-k-means++', n_init = 10)
kmeans.fit(df_ftir.iloc[:, 5:])
df_ftir.loc[:, 'cluster'] = kmeans.labels_

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.stripplot(x = 'family', y = 'cluster', hue = 'instance', dodge = True, data = df_ftir, ax = ax)
plt.title('Distribution of clusters per rock family before and after')
plt.savefig(os.path.join(figspath, 'all-rocks_kmeans_stripplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
# fig, ax = plt.subplots(figsize = (24, 12))
# sns.kdeplot(x = 'l_max_peak', y = 'cluster', hue = 'family', shade = True, alpha = 0.5, data = df_ftir.query("instance == 'before'"), ax = ax, 
#             palette = 'deep')

In [ ]:
# fig, ax = plt.subplots(figsize = (24, 12))
# sns.kdeplot(x = 'l_max_peak', y = 'cluster', hue = 'family', shade = True, alpha = 0.5, data = df_ftir.query("instance == 'after'"), ax = ax, 
#             palette = 'deep')

In [ ]:
# fig, ax = plt.subplots(figsize = (12, 12))
# sns.violinplot(x = 'family', y = 'cluster', hue = 'instance', data = df_ftir, split = True, orient = 'v', inner = 'quartile', scale = 'area', ax = ax)
# plt.yticks(range(6), range(6))
# plt.title('Cluter distribution pero rock family before and after ')
# plt.savefig(os.path.join(figspath, 'all-rocks_kmeans_violinplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
left_cols = df_ftir.columns[:5].to_list() + ['cluster']

In [ ]:
perplexity = p = 50
neighbors = int(3 * p)
tsne = TSNE(n_components = 2, n_iter = 10000, angle = 0.8, n_neighbors = neighbors, perplexity = p, random_state = 5, learning_rate = 10)
X_hat = tsne.fit_transform(df_ftir.iloc[:, 5:-1].values)

In [ ]:
x_hat = pd.concat([df_ftir.loc[:, left_cols], pd.DataFrame(X_hat, columns = ['u', 'v'])], axis = 1)
fig, ax = plt.subplots(ncols = 2, figsize = (24, 13), sharey = True)
plt.subplots_adjust(wspace = 0.05)
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'before'"), ax = ax[0], palette = 'viridis', legend = 'full')
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'after' "), ax = ax[1], palette = 'viridis', legend = 'full')
ax[0].set_title('before')
ax[1].set_title('after')
fig.suptitle('tSNE before & after for all samples, perplexity = ' + str(p), fontsize = 12, fontweight = 'bold', y = 0.92)
plt.savefig(os.path.join(figspath, 'all-rocks_tsne.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
mask = df.loc[:, pp.ftir_cols].isna().any(axis = 1) == False
ix = ix_before_and_after(df.loc[:, pp.meta_cols], mask = mask, subset = ['before', 'heat_treatment', 'perf'])
df_ftir_bna = df_ftir.set_index('index').loc[ix, :].copy()
df_ftir_bna.reset_index(drop = False, inplace = True)

In [ ]:
# data = df_ftir_bna.loc[:, ['instance'] + list(pp.ftir_lambdas)]
# data_before = data.copy()
# data_before.loc[data.instance == 'after', 'instance':] = np.nan
# fig, ax = plt.subplots(figsize = (12, 12))
# sns.heatmap(data_before.iloc[:, 1:], cmap = 'viridis', vmin = 0, vmax = 1)
# plt.title('FTIR data')

In [ ]:
# data_after = data.copy()
# data_after.loc[data.instance == 'before', 'instance':] = np.nan
# fig, ax = plt.subplots(figsize = (12, 12))
# sns.heatmap(data_after.iloc[:, 1:], cmap = 'viridis', vmin = 0, vmax = 1)
# plt.title('FTIR data')

In [ ]:
nclusters = 10
kmeans_bna = KMeans(n_clusters = nclusters, max_iter = 10000, init = 'scalable-k-means++', n_init = 10)
kmeans_bna.fit(df_ftir_bna.iloc[:, 5:-1])
df_ftir_bna.loc[:, 'cluster'] = kmeans_bna.labels_

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.stripplot(x = 'family', y = 'cluster', hue = 'instance', dodge = True, data = df_ftir_bna, ax = ax)
plt.yticks(range(nclusters), range(nclusters))
plt.title('Distribution of clusters per rock family before and after')
plt.savefig(os.path.join(figspath, 'all-rocks-bna_kmeans_stripplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.violinplot(x = 'family', y = 'cluster', hue = 'instance', data = df_ftir_bna, split = True, orient = 'v', inner = 'quartile', scale = 'area', ax = ax)
plt.yticks(range(nclusters), range(nclusters))
plt.title('Cluter distribution pero rock family before and after ')
plt.savefig(os.path.join(figspath, 'all-rocks-bna_kmeans_violinplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
perplexity = p = 30
neighbors = int(5 * p)
tsne = TSNE(n_components = 2, n_iter = 10000, angle = 0.8, n_neighbors = neighbors, perplexity = p, random_state = 5, learning_rate = 10, method = 'exact')

X_hat = tsne.fit_transform(df_ftir_bna.iloc[:, 5:-1].values)

In [ ]:
x_hat = pd.concat([df_ftir_bna.loc[:, left_cols], pd.DataFrame(X_hat, columns = ['u', 'v'])], axis = 1)
fig, ax = plt.subplots(ncols = 2, figsize = (24, 12.1), sharey = True)
plt.subplots_adjust(wspace = 0.05)

sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'before'"), ax = ax[0], palette = 'viridis', legend = 'full')
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'after' "), ax = ax[1], palette = 'viridis', legend = 'full')
ax[0].set_title('before')
ax[1].set_title('after')
fig.suptitle('tSNE before & after for all samples, perplexity = ' + str(p), fontsize = 12, fontweight = 'bold', y = 0.92)
plt.savefig(os.path.join(figspath, 'all-rocks-bna_tsne.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
df_temp = df_ftir.query("tag == 'sh_001'").copy()
df_temp.reset_index(drop = True, inplace = True)

In [ ]:
nclusters = 4
kmeans_sh = KMeans(n_clusters = nclusters, max_iter = 10000, init = 'scalable-k-means++', n_init = 10)
kmeans_sh.fit(df_temp.iloc[:, 5:-1])
df_temp.loc[:, 'cluster'] = kmeans_sh.labels_

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.stripplot(x = 'instance', y = 'cluster', hue = 'cluster', dodge = False, data = df_temp, ax = ax)
plt.yticks(range(nclusters), range(nclusters))
plt.title('Cluter distribution before and after for shale sample sh_1')
plt.savefig(os.path.join(figspath, 'sh-1_kmeans_stripplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.violinplot(x = 'family', y = 'cluster', hue = 'instance', data = df_temp, split = True, orient = 'v', inner = 'quartile', scale = 'count', ax = ax)
plt.xlabel('')
plt.xticks([0], [''])
plt.yticks([0, 1, 2, 3], [0, 1, 2, 3])
plt.title('Cluter distribution before and after for shale sample sh_1')
plt.savefig(os.path.join(figspath, 'sh-1_kmeans_violinplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
perplexity = p = 50
neighbors = int(3 * p)
print('neighbors = %d' % (neighbors))
tsne = TSNE(n_components = 2, n_iter = 10000, angle = 0.8, n_neighbors = neighbors, perplexity = p, random_state = 5, learning_rate = 10, method = 'exact')

X_hat = tsne.fit_transform(df_temp.iloc[:, 5:-1].values)

In [ ]:
x_hat = pd.concat([df_temp.loc[:, left_cols], pd.DataFrame(X_hat, columns = ['u', 'v'])], axis = 1)

fig, ax = plt.subplots(ncols = 2, figsize = (24, 13), sharey = True)
plt.subplots_adjust(wspace = 0.05)
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'before'"), ax = ax[0], palette = 'viridis', legend = 'full')
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'after' "), ax = ax[1], palette = 'viridis', legend = 'full')
ax[0].set_title('before')
ax[1].set_title('after')
fig.suptitle('tSNE before & after for shale sample sh_1, perplexity = ' + str(p), fontsize = 12, fontweight = 'bold', y = 0.92)
plt.savefig(os.path.join(figspath, 'sh-1_tsne.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
df_temp = df_ftir.query("family == 'shale'").copy()
df_temp.reset_index(drop = True, inplace = True)

In [ ]:
nclusters = 10
kmeans_sh = KMeans(n_clusters = nclusters, max_iter = 10000, init = 'scalable-k-means++', n_init = 10)
kmeans_sh.fit(df_temp.iloc[:, 5:-1])
df_temp.loc[:, 'cluster'] = kmeans_sh.labels_

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.stripplot(x = 'instance', y = 'cluster', hue = 'cluster', dodge = False, data = df_temp, ax = ax)
plt.yticks(range(nclusters), range(nclusters))
plt.title('Cluster distribution before and after for shale samples')
plt.savefig(os.path.join(figspath, 'shale_kmeans_stripplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.violinplot(x = 'family', y = 'cluster', hue = 'instance', data = df_temp, split = True, orient = 'v', inner = 'quartile', scale = 'count', ax = ax)
plt.xlabel('')
plt.xticks([0], [''])
plt.yticks(range(nclusters), range(nclusters))
plt.title('Cluster distribution before and after for shale samples')
plt.savefig(os.path.join(figspath, 'shale_kmeans_violinplot.png'), dpi = 300, bbox_inches = 'tight')

In [ ]:
perplexity = p = 50
neighbors = int(3 * p)
tsne = TSNE(n_components = 2, n_iter = 10000, angle = 0.8, n_neighbors = neighbors, perplexity = p, random_state = 5, learning_rate = 10, method = 'exact')
X_hat = tsne.fit_transform(df_temp.iloc[:, 5:-1].values)

In [ ]:
x_hat = pd.concat([df_temp.loc[:, left_cols], pd.DataFrame(X_hat, columns = ['u', 'v'])], axis = 1)

fig, ax = plt.subplots(ncols = 2, figsize = (24, 13), sharey = True)
plt.subplots_adjust(wspace = 0.05)
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'before'"), ax = ax[0], palette = 'viridis', legend = 'full')
sns.scatterplot(x = 'u', y = 'v', hue = 'cluster', data = x_hat.query("instance == 'after' "), ax = ax[1], palette = 'viridis', legend = 'full')
ax[0].set_title('before')
ax[1].set_title('after')
fig.suptitle('tSNE before & after for shale samples, perplexity = ' + str(p), fontsize = 12, fontweight = 'bold', y = 0.92)
plt.savefig(os.path.join(figspath, 'shale_tsne.png'), dpi = 300, bbox_inches = 'tight')